In [1]:
import keras
from src.ParticleNet.PointNet import PointNetLayer
from src.ParticleNet.GlobalPooling import GlobalAveragePoolingLayer
from utilities import utilities
from ML.ParticleCloud.data_loaders import load_data_qg_tagging

#### Loading the Data

In [2]:
X_train, y_train = load_data_qg_tagging(quark_data_path="../../../Data/Trainning/q_jets.csv", gluon_data_path="../../../Data/Trainning/g_jets.csv")
X_val, y_val = load_data_qg_tagging(quark_data_path="../../../Data/Validation/q_jets.csv", gluon_data_path="../../../Data/Validation/g_jets.csv")

In [3]:
print(f"Size of the Training set: {len(X_train)}")
print(f"Size of the Validation set: {len(X_val)}")

Size of the Training set: 251379
Size of the Validation set: 44362


#### Setting up the NN

In [4]:
# MLP for the first conv layer
mlp_jets = keras.Sequential([
    keras.layers.InputLayer(shape=[4]),
    keras.layers.Dense(32),
    keras.layers.ReLU(),
    keras.layers.Dense(32),
    keras.layers.ReLU()
])

mlp_jets2 = keras.Sequential([
    keras.layers.InputLayer(shape=[32]),
    keras.layers.Dense(64),
    keras.layers.ReLU(),
    keras.layers.Dense(64),
    keras.layers.ReLU()
])

# MLP for the second convolutional layer
jet_tag_model = keras.Sequential([
    keras.layers.InputLayer(shape=(30, 5)),
    PointNetLayer(mlp=mlp_jets, output_dim=32),
    PointNetLayer(mlp=mlp_jets2, output_dim=64),
    GlobalAveragePoolingLayer(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(rate=0.1),
    keras.layers.Dense(2, activation='softmax')
])

jet_tag_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ point_net_layer (PointNetLayer) │ (None, 30, 33)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ point_net_layer_1               │ (None, 30, 65)         │         6,272 │
│ (PointNetLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling_layer    │ (None, 64)             │             0 │
│ (GlobalAveragePoolingLayer)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,066 (62.76 KB)

 Trainable params: 16,066 (62.76 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
jet_tag_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
history = jet_tag_model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))   

Epoch 1/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.7201 - loss: 0.5432 - val_accuracy: 0.7542 - val_loss: 0.5018
Epoch 2/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.7540 - loss: 0.5046 - val_accuracy: 0.7613 - val_loss: 0.4938
Epoch 3/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.7582 - loss: 0.4969 - val_accuracy: 0.7483 - val_loss: 0.5032
Epoch 4/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.7641 - loss: 0.4908 - val_accuracy: 0.7739 - val_loss: 0.4765
Epoch 5/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.7683 - loss: 0.4844 - val_accuracy: 0.7780 - val_loss: 0.4725
Epoch 6/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.7697 - loss: 0.4808 - val_accuracy: 0.7806 - val_loss: 0.4745
Epoch 7/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.7719 - loss: 0.4796 - val_accuracy: 0.7689 - val_loss: 0.4809
Epoch 8/100
3928/3928 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.7730 -

KeyboardInterrupt: 

In [ ]:
# perfoming the prediction
y_train_pred = jet_tag_model.predict(X_train)
y_val_pred = jet_tag_model.predict(X_val)

In [ ]:
print("Trainnig set:")
utilities.display_metrics(y_true=y_train[:, 0], y_pred=y_train_pred[:, 0] > 0.5)
print("Validation set:")
utilities.display_metrics(y_true=y_val[:, 0], y_pred=y_val_pred[:, 0] > 0.5)

In [ ]:
utilities.display_roc_curve(y_true=y_val[:, 0], y_score=y_val_pred[:, 0])

In [ ]:
utilities.plot_hist_trainning(history)

In [ ]:
utilities.save_model(jet_tag_model, history, "QuarkGluon_Tagging_PointNet")